### Quantum computation with Qiskit | Introduction to errors and error mitigation

#### Introduction

Qiskit Composer is great and all but for most realistic projects you will be working on, you need to write at least some code.

Let's use the code that the Composer generated for us to create what's called a uniform superposition of the computational basis states for a $3$-qubit system. We'll also need a classical register to store the results once we have the binary computational basis outcomes.

In [ ]:
from qiskit import QuantumCircuit

circ = QuantumCircuit(3, 3) # 3 qubits, 3 classical bits
for i in range(3):
    circ.h(i)

In [ ]:
circ.draw(output='mpl') # mpl uses the matplotlib library to draw the circuit instead of the default text-based output

The circuit will evolve the initial state |000> to H_3|000> where H_3 is the Hadamard gate applied to all three qubits or the tensor product of three Hadamard gates. What do you think will be the output of this circuit?

While the state does evolve, seeing what it evolves to requires observing the state. This is done by measuring the state in the computational basis (you can measure states in other bases too, but let's focus on the computational basis for now). The measurement process is probabilistic, so we will need to run the circuit multiple times to get a good idea of what the probability distribution of the outcomes is.

`shots` is the number of times the circuit is run. Generally, the more shots, the more accurate the results will be.

Let's add measurement operators to the circuit and run it for 1024 shots.
`QuantumCircuit`s have a `measure` method that takes the qubit to measure and the classical bit to store the result in. Alternatively if we want to measure all qubits, we can use the `measure_all` method.

In [ ]:
circ.measure_all(add_bits=False) # or circ.measure([0,1,2], [0,1,2]) to measure qubits 0, 1, and 2 and store the results in classical bits 0, 1, and 2
circ.draw(output='mpl')

### Running circuits

You can run the circuits on the real quantum devices if you really want to, but you'll have to wait in the queue. For the purposes of this demonstration, we'll stick to simulating things on our computers.

Qiskit (more specifically `qiskit-ibm-runtime`) provides us with `FakeProvider` which allows us to mimic and simulate the behavior of real devices through the snapshots they provide.

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeKyoto

backend = FakeKyoto()

shots = 1024 # Let's not forget the number of shots

job = backend.run(circ, shots=shots) # For a real device, you'd be waiting in queue here
result = job.result()

In [ ]:
result

Seems like there's a bunch of information within the result object. We really just want the probabilities of the basis states to decipher what happened:

In [ ]:
counts = result.get_counts()
probabilities = {state: count / shots for state, count in counts.items()}
probabilities

See how these probabilities are not exactly ideal? This is because of errors in the quantum computation. These errors can be due to a variety of reasons, such as noise in the system, imperfect gates, small number of shots and so on. We can't get rid of these errors, but we can try to mitigate them. More on this later. Let's try to visualize the results first.

In [ ]:
from qiskit.visualization import plot_histogram

plot_histogram(probabilities)

What if we did want the ideal state that the circuit evolved to? Well Qiskit Aer has ideal simulators that we can use. Qiskit Aer offers high performance simulators for simulations. It also supports noise models for realistic simulations. We can also tweak the noise models and see how they affect the results.

In [ ]:
from qiskit_aer import AerSimulator

circ = QuantumCircuit(3, 3)
for i in range(3):
    circ.h(i)
circ.save_statevector()

aer_backend = AerSimulator(method='statevector')
job = aer_backend.run(circ)

result = job.result()
statevector = result.get_statevector()
statevector

In [ ]:
statevector.draw('latex')

#### Note: Estimator and Sampler

Since Qiskit 1.0 (recently released), backend.run methods have been deprecated. Instead, you have to rely on `Sampler` and `Estimator` primitives to do the job for you - they're really an abstraction of the processes we just carried out. For the most part, whatever we do can essentially be done with the `Sampler` primitive. The `Estimator` primitive is used to estimate the expectation value of an observable. The interface is still pretty much the same, so you shouldn't have any trouble transitioning.

See https://docs.quantum.ibm.com/guides/get-started-with-primitives for a guide on how to use these primitives.

##### Note: Transpiling circuits

The circuit we've run so far are very simple. Complicated circuits still need to respect the topology of the device they're running on - some gates might not be available for some qubits, some qubits might not be connected to each other etc. Additionally, some circuits could have parts that are redundant or can be optimized. Some circuits have gates that need to be decomposed into a series of operations that are available on the device. The process that covers all of this is called transpilation.

Circuits are abstract until they're converted into physical/ISA circuits using a transpiler.

Beyond simulations, ensure you transpile your circuits before running them on real devices. The highest level and perhaps the easiest to use function for this is `transpile`.

In [ ]:
backend.operation_names # (Basis) Operations available on the FakeKyoto backend

In [ ]:
test_circuit = QuantumCircuit(2, 2)
test_circuit.y(0)

test_circuit.draw('mpl')

In [ ]:
from qiskit import transpile

transpiled_circuit = transpile(test_circuit, backend)
transpiled_circuit.draw('mpl')

### A taste of optimization: preparing an entangled state

In [ ]:
# The target state is |0> + |1>
# We can use the statevector simulator to get the statevector of the circuit and then calculate the fidelity of the statevector with the target statevector.
import numpy as np
from qiskit.quantum_info import Statevector

target_state = np.array([1, 0, 0, 1])/np.sqrt(2)
target_statevector = Statevector(target_state)
target_statevector.draw('latex')

In [ ]:
import numpy as np
from qiskit.circuit import Parameter
from qiskit.quantum_info import state_fidelity
from scipy.optimize import minimize
from qiskit_aer import StatevectorSimulator

param_a = Parameter('a')
param_b = Parameter('b')

qc = QuantumCircuit(2)
qc.ry(Parameter('a'), 0)
qc.rx(Parameter('b'), 0)
qc.cx(0, 1)

def objective_function(params):
    aer_backend = StatevectorSimulator()
    bound_circuit = qc.assign_parameters({'a': params[0], 'b': params[1]})
    job = aer_backend.run(bound_circuit)
    result = job.result()
    statevector = result.get_statevector()
    fidelity = state_fidelity(target_statevector, statevector)
    return 1 - fidelity

initial_guess = [0,  0] # Try changing and varying the initial guess to see what other values the optimizer converges to
result = minimize(objective_function, initial_guess, method='BFGS', tol=1e-8) # Try changing the optimization method and tolerance to see how the optimizer behaves

In [ ]:
result

In [ ]:
# The optimized parameters are stored in the x attribute of the result object, see what the state is
optimized_params = result.x
optimized_circuit = qc.assign_parameters({'a': optimized_params[0], 'b': optimized_params[1]})
aer_backend = StatevectorSimulator()
job = aer_backend.run(optimized_circuit)
res = job.result()
optimized_statevector = res.get_statevector()
optimized_statevector.draw('latex')


In [ ]:
plot_histogram(optimized_statevector.sample_counts(shots=2048))

In [ ]:
hadamard_circuit = QuantumCircuit(1)
hadamard_circuit.h(0)
transpile(hadamard_circuit, basis_gates=['ry', 'rx']).draw('mpl')

Why pi? What's going on?

### Errors and error mitigation

Intro to errors covered in the slides

#### Readout error mitigation

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakePerth
aer_perth_simulator = AerSimulator.from_backend(FakePerth())
# from_backend creates a simulator and takes noise parameters from the backend
# Could also create own custom NoiseModel

from qiskit.circuit.random import random_circuit

example_circuit = random_circuit(3, depth=3, measure=True)
example_circuit.draw('mpl')

In [ ]:
calibration_circuits = []
# Create circuits with all possible initial states for 3 qubits and measure them
for i in range(2**3):
    qc = QuantumCircuit(3, 3)
    binary = reversed(format(i, '03b')) # reversed because qiskit uses the little-endian convention
    for j, bit in enumerate(binary):
        if bit == '1':
            qc.x(j)
    qc.measure_all(add_bits=False)
    calibration_circuits.append(qc)

In [ ]:
calibration_circuits[2].draw('mpl')

In [ ]:
shots = 2048

In [ ]:
# Run the calibration circuits on the simulator

calibration_dict = {}

for i, qc in enumerate(calibration_circuits):
    job = aer_perth_simulator.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Given initial state', format(i, '03b'), 'we measured', counts)
    calibration_dict[format(i, '03b')] = counts

In [ ]:
# Now we can form the calibration matrix
calibration_matrix = np.zeros((2**3, 2**3))

for i in range(2**3):
    for j in range(2**3):
        calibration_matrix[i, j] = calibration_dict[format(i, '03b')].get(format(j, '03b'), 0) / shots

In [ ]:
# Let's see what the calibration matrix looks like using a heatmap
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
sns.heatmap(calibration_matrix, annot=True, cmap='viridis', cbar=False)
plt.xlabel('Measured state')
plt.ylabel('Given state')
plt.title('Calibration matrix')
plt.show()

In [ ]:
# Now we can use the calibration matrix to correct the results of the example circuit
example_circuit_transpiled = transpile(example_circuit, backend=aer_perth_simulator) # Try commenting this line out and see what happens: migh
job = aer_perth_simulator.run(example_circuit_transpiled, shots=shots)
result = job.result()
counts = result.get_counts()

# invert the calibration matrix
inverse_calibration_matrix = np.linalg.inv(calibration_matrix)

# Form the counts_vector from the counts dictionary
counts_vector = np.array([counts.get(format(i, '03b'), 0) for i in range(2**3)])

# Correct the counts_vector
corrected_counts_vector = inverse_calibration_matrix @ counts_vector

# Form the corrected counts dictionary
corrected_counts = {format(i, '03b'): round(corrected_counts_vector[i]) for i in range(2**3)}

old_probabilities = {state: count / shots for state, count in counts.items()}
new_probabilities = {state: count / shots for state, count in corrected_counts.items()}


# Plot the corrected counts versus the original probabilities (side by side with the value above the bars)
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
plot_histogram(old_probabilities, ax=ax[0], title='Original probabilities')
plot_histogram(new_probabilities, ax=ax[1], title='Corrected probabilities')
plt.show()

In [ ]:
example_circuit_state = example_circuit.remove_final_measurements(inplace=False)
Statevector.from_instruction(example_circuit_state).probabilities_dict()

### Exercises

**Noise due to sampling:** How can you quantify the variability of the results as a function of the number of shots?

**Eigenstates and eigenvalues of different bases:** Measuring in the computational basis required just using a measurement method. The computational basis corresponds to the eigenstates of the Pauli-Z operator. What if we wanted to measure in a different basis? How would you get measurement results in say the X-basis?

**Generalizing the structure for controlled unitaries:** What would be the controlled version of the Y gate in matrix form? What about the controlled version of a general 2x2 unitary matrix?

**Trying out the real devices:** Try and send your circuits to a real device. What do you get? Do you think you can mitigate the readout errors using the error mitigation technique we discussed?

**Calibration Matrices:** Are there any problems with the inverse matrix technique we discussed for readout error mitigation? Can you think of a better way to mitigate readout errors using the calibration matrix?

**A custom experiment:** Could you try to recreate the experiment described during the intro to errors section? More specifically, create circuits with varying depth d, apply the X gate d number of times, carry out measurements and calculate the expectation value of Z.
Hint: The counts will already be in the computational basis, so you don't need to do anything special to get them in the right basis.

**Extra:** Try to understand the concept of decoherence and how it affects quantum computation

### References

- Nielsen and Chuang is a given and a great resource for quantum computing
- Qiskit Summer School videos 
    - 2021 Qiskit Summer School videos on types of noise https://www.youtube.com/watch?v=kV0RsKXSqRg
    - Associated resources: https://github.com/Qiskit/platypus/blob/main/notebooks/summer-school/2021
- [Refresher and Math] Quantum Computing for Computer Scientists: https://www.youtube.com/watch?v=F_Riqjdh2oM
- Running on the real devices (check the README): https://github.com/Qiskit/qiskit-ibm-runtime
    - Make sure you make an IBM Quantum account and get your API token!
- If you run out of resources, or have any questions, feel free to ask in the Slack!